In [76]:
import pandas as pd
import numpy as np
import math
%matplotlib inline
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [80]:
df_train_1 = pd.read_csv("../feature/pv_uv_train_fea.txt", sep='\t')
df_train_2 = pd.read_csv('../feature/poi_info_train_fea_2.txt', sep='\t')
df_train_3 = pd.read_csv('../feature/deal_train_fea.txt', sep='\t')
df_train_4 = pd.read_csv('../feature/user_poi_rec_dish_train_fea.txt', sep='\t')
df_train_label = pd.read_csv('../raw_data/deal_sales_train.txt', sep='\t')

df_tmp = pd.merge(df_train_1, df_train_2, on=['deal_id', 'poi_id'])
df_tmp = pd.merge(df_tmp, df_train_3, on=['deal_id', 'poi_id'])
df_tmp = pd.merge(df_tmp, df_train_4, on=['deal_id', 'poi_id'])

df_final = pd.merge(df_tmp, df_train_label, on=['deal_id', 'poi_id'])

In [83]:
df_final["sales"] = np.log(df_final["sales"])

def process(final):
    final["original_marketPrice_discount"] = final["dishes_price"] - final["market_price"]
    final["original_salePrice_discount"] = final["dishes_price"] - final["price"]
    final["original_marketPrice_discount_rate"] = final["original_marketPrice_discount"]/final["dishes_price"]
    final["original_salePrice_discount_rate"] = final["original_salePrice_discount"]/final["dishes_price"]
    return final

df_final = process(df_final)

In [103]:
len(df_final.columns.values.tolist())

118

In [86]:
del df_final['deal_id']
del df_final['poi_id']

In [97]:
df_test_1 = pd.read_csv("../feature/pv_uv_test_fea.txt", sep='\t')
df_test_2 = pd.read_csv('../feature/poi_info_test_fea_2.txt', sep='\t')
df_test_3 = pd.read_csv('../feature/deal_test_fea.txt', sep='\t')
df_test_4 = pd.read_csv('../feature/user_poi_rec_dish_test_fea.txt', sep='\t')
df_test_label = pd.read_csv('../raw_data/deal_poi_test.txt', sep='\t')

df_tmp_te = pd.merge(df_test_1, df_test_2, on=['deal_id', 'poi_id'])
df_tmp_te = pd.merge(df_tmp_te, df_test_3, on=['deal_id', 'poi_id'])
df_tmp_te = pd.merge(df_tmp_te, df_test_4, on=['deal_id', 'poi_id'])
df_final_te = pd.merge(df_tmp_te, df_test_label, on=['deal_id', 'poi_id'])

df_final_te = process(df_final_te)

In [98]:
# df_final_te.count()

In [99]:
del df_final_te['deal_id']
del df_final_te['poi_id']

In [89]:
y = df_final.sales
X = df_final.drop(['sales'], axis=1)
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=7)

# This is baseline

In [90]:
mean_train = np.mean(train_y)
baseline_prediction = np.ones(test_y.shape)*mean_train
mae_baseline = mean_absolute_error(baseline_prediction, test_y)
print "mae_baseline is %f"%mae_baseline

mae_baseline is 0.907866


## 使用XgBoost进行训练

In [72]:
from xgboost import XGBRegressor
import xgboost as xgb

In [104]:
my_model = XGBRegressor(n_estimators=1500, max_depth=5, learning_rate=0.15, n_jobs=4)
my_model.fit(train_X, train_y, early_stopping_rounds=5, 
             eval_set=[(test_X, test_y)], verbose=True, eval_metric='mae')

[0]	validation_0-mae:2.64371
Will train until validation_0-mae hasn't improved in 5 rounds.
[1]	validation_0-mae:2.24715
[2]	validation_0-mae:1.91007
[3]	validation_0-mae:1.62395
[4]	validation_0-mae:1.38386
[5]	validation_0-mae:1.19078
[6]	validation_0-mae:1.04163
[7]	validation_0-mae:0.930502
[8]	validation_0-mae:0.850081
[9]	validation_0-mae:0.794507
[10]	validation_0-mae:0.756919
[11]	validation_0-mae:0.732792
[12]	validation_0-mae:0.716678
[13]	validation_0-mae:0.706549
[14]	validation_0-mae:0.699404
[15]	validation_0-mae:0.69522
[16]	validation_0-mae:0.692407
[17]	validation_0-mae:0.690466
[18]	validation_0-mae:0.689762
[19]	validation_0-mae:0.689061
[20]	validation_0-mae:0.688577
[21]	validation_0-mae:0.688289
[22]	validation_0-mae:0.688194
[23]	validation_0-mae:0.687715
[24]	validation_0-mae:0.68807
[25]	validation_0-mae:0.687253
[26]	validation_0-mae:0.686509
[27]	validation_0-mae:0.685505
[28]	validation_0-mae:0.685296
[29]	validation_0-mae:0.68544
[30]	validation_0-mae:0.684

[259]	validation_0-mae:0.639539
[260]	validation_0-mae:0.639483
[261]	validation_0-mae:0.639373
[262]	validation_0-mae:0.639321
[263]	validation_0-mae:0.639271
[264]	validation_0-mae:0.639261
[265]	validation_0-mae:0.639195
[266]	validation_0-mae:0.639101
[267]	validation_0-mae:0.639092
[268]	validation_0-mae:0.639034
[269]	validation_0-mae:0.639018
[270]	validation_0-mae:0.638935
[271]	validation_0-mae:0.638926
[272]	validation_0-mae:0.638916
[273]	validation_0-mae:0.638837
[274]	validation_0-mae:0.638804
[275]	validation_0-mae:0.638742
[276]	validation_0-mae:0.638648
[277]	validation_0-mae:0.638455
[278]	validation_0-mae:0.638398
[279]	validation_0-mae:0.638359
[280]	validation_0-mae:0.638276
[281]	validation_0-mae:0.638242
[282]	validation_0-mae:0.638217
[283]	validation_0-mae:0.638168
[284]	validation_0-mae:0.638159
[285]	validation_0-mae:0.638046
[286]	validation_0-mae:0.637906
[287]	validation_0-mae:0.63775
[288]	validation_0-mae:0.637568
[289]	validation_0-mae:0.637463
[290]	val

[516]	validation_0-mae:0.627818
[517]	validation_0-mae:0.627807
[518]	validation_0-mae:0.62772
[519]	validation_0-mae:0.627708
[520]	validation_0-mae:0.627676
[521]	validation_0-mae:0.627631
[522]	validation_0-mae:0.627624
[523]	validation_0-mae:0.627606
[524]	validation_0-mae:0.627586
[525]	validation_0-mae:0.627577
[526]	validation_0-mae:0.627575
[527]	validation_0-mae:0.627563
[528]	validation_0-mae:0.627523
[529]	validation_0-mae:0.627464
[530]	validation_0-mae:0.627408
[531]	validation_0-mae:0.627311
[532]	validation_0-mae:0.627279
[533]	validation_0-mae:0.62725
[534]	validation_0-mae:0.627232
[535]	validation_0-mae:0.627195
[536]	validation_0-mae:0.627181
[537]	validation_0-mae:0.627137
[538]	validation_0-mae:0.627056
[539]	validation_0-mae:0.627041
[540]	validation_0-mae:0.627027
[541]	validation_0-mae:0.627014
[542]	validation_0-mae:0.626998
[543]	validation_0-mae:0.626988
[544]	validation_0-mae:0.626966
[545]	validation_0-mae:0.626965
[546]	validation_0-mae:0.626962
[547]	vali

[773]	validation_0-mae:0.621711
[774]	validation_0-mae:0.621705
[775]	validation_0-mae:0.621677
[776]	validation_0-mae:0.621647
[777]	validation_0-mae:0.621646
[778]	validation_0-mae:0.621632
[779]	validation_0-mae:0.621626
[780]	validation_0-mae:0.621617
[781]	validation_0-mae:0.621577
[782]	validation_0-mae:0.621551
[783]	validation_0-mae:0.621526
[784]	validation_0-mae:0.621478
[785]	validation_0-mae:0.621463
[786]	validation_0-mae:0.62145
[787]	validation_0-mae:0.621452
[788]	validation_0-mae:0.621443
[789]	validation_0-mae:0.621407
[790]	validation_0-mae:0.621375
[791]	validation_0-mae:0.621349
[792]	validation_0-mae:0.621305
[793]	validation_0-mae:0.621303
[794]	validation_0-mae:0.621261
[795]	validation_0-mae:0.621246
[796]	validation_0-mae:0.621226
[797]	validation_0-mae:0.62121
[798]	validation_0-mae:0.621208
[799]	validation_0-mae:0.621191
[800]	validation_0-mae:0.62119
[801]	validation_0-mae:0.621187
[802]	validation_0-mae:0.621177
[803]	validation_0-mae:0.621177
[804]	valid

[1029]	validation_0-mae:0.617435
[1030]	validation_0-mae:0.61739
[1031]	validation_0-mae:0.617369
[1032]	validation_0-mae:0.617352
[1033]	validation_0-mae:0.617336
[1034]	validation_0-mae:0.617342
[1035]	validation_0-mae:0.617334
[1036]	validation_0-mae:0.617322
[1037]	validation_0-mae:0.61727
[1038]	validation_0-mae:0.617255
[1039]	validation_0-mae:0.617244
[1040]	validation_0-mae:0.617226
[1041]	validation_0-mae:0.617198
[1042]	validation_0-mae:0.617193
[1043]	validation_0-mae:0.617176
[1044]	validation_0-mae:0.617161
[1045]	validation_0-mae:0.617074
[1046]	validation_0-mae:0.617055
[1047]	validation_0-mae:0.61703
[1048]	validation_0-mae:0.617028
[1049]	validation_0-mae:0.617011
[1050]	validation_0-mae:0.617008
[1051]	validation_0-mae:0.616993
[1052]	validation_0-mae:0.616983
[1053]	validation_0-mae:0.616983
[1054]	validation_0-mae:0.616985
[1055]	validation_0-mae:0.616978
[1056]	validation_0-mae:0.616977
[1057]	validation_0-mae:0.616962
[1058]	validation_0-mae:0.616874
[1059]	valida

[1278]	validation_0-mae:0.614294
[1279]	validation_0-mae:0.614283
[1280]	validation_0-mae:0.614264
[1281]	validation_0-mae:0.614258
[1282]	validation_0-mae:0.614241
[1283]	validation_0-mae:0.614231
[1284]	validation_0-mae:0.614225
[1285]	validation_0-mae:0.61422
[1286]	validation_0-mae:0.614216
[1287]	validation_0-mae:0.614219
[1288]	validation_0-mae:0.614208
[1289]	validation_0-mae:0.614185
[1290]	validation_0-mae:0.614167
[1291]	validation_0-mae:0.614161
[1292]	validation_0-mae:0.61416
[1293]	validation_0-mae:0.61416
[1294]	validation_0-mae:0.614148
[1295]	validation_0-mae:0.614131
[1296]	validation_0-mae:0.614136
[1297]	validation_0-mae:0.614116
[1298]	validation_0-mae:0.6141
[1299]	validation_0-mae:0.614081
[1300]	validation_0-mae:0.614063
[1301]	validation_0-mae:0.614051
[1302]	validation_0-mae:0.614042
[1303]	validation_0-mae:0.614028
[1304]	validation_0-mae:0.614018
[1305]	validation_0-mae:0.614002
[1306]	validation_0-mae:0.614002
[1307]	validation_0-mae:0.613983
[1308]	validati

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.15, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=1500,
       n_jobs=4, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [107]:
test_pred = my_model.predict(df_final_te)

## 预测结果并提交

In [109]:
df_test_label = pd.read_csv('../raw_data/deal_poi_test.txt', sep='\t')
def gen_deal_poi(deal_id, poi_id):
    return str(deal_id) + '_' + str(poi_id)
df_test_label['deal_poi'] = df_test_label.apply(lambda row : gen_deal_poi(row['deal_id'], row['poi_id']), axis=1)
df_test_label['sales'] = test_pred
del df_test_label['deal_id']
del df_test_label['poi_id']
df_test_label['sales'] = np.exp(df_test_label['sales'])
df_test_label.to_csv('../result/first_res.csv', sep=',', index=False)
# df_test_label.count()